In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from hyperopt import fmin, tpe, hp

In [ ]:
%pip install hyperopt

In [ ]:
%pip install tensorflow

In [ ]:
%pip install keras

In [4]:
# Load Dataset into the Data Frame
df = pd.read_csv('../data/gemstone_price.csv')

In [5]:
df

,Date,carat,cut,color,clarity,x,y,z,price,GemstoneNames
0,01/01/2018,0.30,Ideal,blue,SI1,4.27,4.29,2.66,499,Spinel
1,01/01/2018,0.33,Premium,yellow,IF,4.42,4.46,2.70,984,Cat's Eye
2,01/01/2018,0.90,Very Good,blue,VVS2,6.04,6.12,3.78,6289,Spinel
3,01/01/2018,0.42,Ideal,purple,VS1,4.82,4.80,2.96,1082,Zircon
4,01/01/2018,0.31,Ideal,purple,VVS1,4.35,4.43,2.65,779,Alexandrite
...,...,...,...,...,...,...,...,...,...,...
26962,10/09/2021,1.11,Premium,yellow,SI1,6.61,6.52,4.09,5408,Cat's Eye
26963,10/09/2021,0.33,Ideal,Green,IF,4.44,4.42,2.74,1114,Topaz
26964,10/09/2021,0.51,Premium,blue,VS2,5.12,5.15,3.17,1656,Sapphire
26965,10/09/2021,0.27,Very Good,purple,VVS2,4.19,4.20,2.60,682,Alexandrite


In [6]:
# Encode categorical features
categorical_columns = ['cut', 'color', 'clarity', 'GemstoneNames']
for column in categorical_columns:
    df[column] = LabelEncoder().fit_transform(df[column])

In [7]:
# Split the data into features and target
X = df.drop(columns=['Date', 'price'])
y = df['price']

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Define a hyperparameter search space
space = {
    'n_hidden_layers': hp.choice('n_hidden_layers', [1, 2, 3]),
    'n_units': hp.choice('n_units', [64, 128, 256]),
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5),
    'epochs': hp.quniform('epochs', 50, 200, 10),
    'batch_size': hp.choice('batch_size', [32, 64, 128])
}

In [10]:
# Define the objective function for hyperparameter tuning
def objective(params):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))
    
    for _ in range(params['n_hidden_layers']):
        model.add(keras.layers.Dense(params['n_units'], activation='relu'))
        model.add(keras.layers.Dropout(params['dropout_rate']))
    
    model.add(keras.layers.Dense(1))  # Output layer for regression
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss='mean_squared_error')
    
    model.fit(X_train, y_train, epochs=int(params['epochs']), batch_size=params['batch_size'], verbose=0)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

In [ ]:
# Perform hyperparameter tuning using Hyperopt
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100)